# Compare Impact-T particle and stat covariance

This is to check the interpretation of the `fort.32` covariance matrix data. Some elements differ from particles in `fort.50`:
- $<x, z>$
- $<x, pz>$
- $<y, z>$
- $<y, pz>$

This is because the stats can be computed at the same time t (`Fladdiag=1`) or drifted to the same z (`Fladdiag=2`).

In [ ]:
from impact import Impact
from impact.lattice import new_write_beam

import numpy as np
from pathlib import Path
from pmd_beamphysics.units import c_light
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

# Easier print
np.set_printoptions(formatter={"float": lambda x: f"{x:6.2e}"}, linewidth=100)


def printsymmat(matrix):
    for i in range(6):
        for j in range(6):
            if j < i:
                print(
                    f"{' ':>8}", end=" "
                )  # Print '-' for redundant lower triangle elements
            else:
                print(
                    f"{matrix[i][j]:8.2f}", end=" "
                )  # Print the number for upper triangle and diagonal
        print()  # New line for each row

# Make data

In [ ]:
def run_impact(drift_to_z=False):
    ifile = "templates/lcls_injector/ImpactT.in"
    I = Impact(ifile, verbose=False)
    I.header["Np"] = 1000
    # Turn Space Charge off.
    I.header["Bcurr"] = 0
    if drift_to_z:
        I.header["Flagdiag"] = 2
    else:
        I.header["Flagdiag"] = 1
    I.stop = 1
    I.numprocs = 0
    # Make a list of s
    for s in np.linspace(0.001, 1, 20):
        ele = new_write_beam(
            s=s, ref_eles=I.lattice
        )  # ref_eles will ensure that there are no naming conflicts
        I.add_ele(ele)
    I.run()
    return I


# Make two sets of data
It = run_impact(drift_to_z=False)
Iz = run_impact(drift_to_z=True)

It.plot()

# Compare drift to z off/on:

In [ ]:
for key in ("cov_x__z", "cov_x__pz", "cov_y__z", "cov_y__pz")[0:1]:
    It.plot(key)

In [ ]:
for key in ("cov_x__z", "cov_x__pz", "cov_y__z", "cov_y__pz")[0:1]:
    Iz.plot(key)

# fort.32 covariance matrix data

Load and label the data

In [ ]:
root = Path(It.path)

In [ ]:
fdat = np.loadtxt(root / "fort.32")[-1, :]
labels = [
    "z",
    "z0avg*xl",
    "xl",
    "sqsum1",
    "xpx",
    "xy",
    "xpy",
    "xz",
    "xpz",
    "sqsum2",
    "ypx",
    "pxpy",
    "zpx",
    "pxpz",
    "sqsum3",
    "ypy",
    "yz",
    "ypz",
    "sqsum4",
    "zpy",
    "pypz",
    "sqsum5",
    "zpz",
    "sqsum6",
]
fstat = {}
for label, dat in zip(labels, fdat):
    fstat[label] = dat

xl = fstat["xl"]
fstat

In [ ]:
F = fstat
fcov = np.array(
    [
        [F["sqsum1"], F["xpx"], F["xy"], F["xpy"], F["xz"], F["xpz"]],
        [0, F["sqsum2"], F["ypx"], F["pxpy"], F["zpx"], F["pxpz"]],
        [0, 0, F["sqsum3"], F["ypy"], F["yz"], F["ypz"]],
        [0, 0, 0, F["sqsum4"], F["zpy"], F["pypz"]],
        [0, 0, 0, 0, F["sqsum5"], F["zpz"]],
        [0, 0, 0, 0, 0, F["sqsum6"]],
    ]
)
# Make the matrix symmetric by copying upper triangle to lower triangle
for i in range(6):
    for j in range(6):
        fcov[j, i] = fcov[i, j]


fcov * 1e9

In [ ]:
print("fort.32 (* 1e9)")
printsymmat(fcov * 1e9)

# Final particles from fort.50

Calculate the covariance matrix from the particles. Note that these aren't exactly at the same final location as the fort.32 file. Here we are looking for similar numbers. 


In [ ]:
pdat = np.loadtxt(root / "fort.50")
pdat.shape

In [ ]:
# Covariance
cov = np.cov(pdat.T)
cov.shape

print("fort.50 with np.cov (* 1e9)")
printsymmat(cov * 1e9)

# LUME-Impact plots

Compare with Lume

In [ ]:
# from impact import Impact
# %config InlineBackend.figure_format = 'retina'
#
# I = Impact('ImpactT.in', workdir = '.', use_temp_dir=False)
# I.load_output()

# I.plot('cov_x__z')

In [ ]:
# I.plot('cov_x__y')

In [ ]:
%%time
Pz = []
for p in It.particles.values():
    p = p.copy()
    p.drift_to_z()
    Pz.append(p)
Pt = []
for p in It.particles.values():
    p = p.copy()
    p.drift_to_t()
    Pt.append(p)
Pz;

In [ ]:
def pzstat(key):
    return np.array([p[key] for p in Pz])


def ptstat(key):
    return np.array([p[key] for p in Pt])

In [ ]:
xkey = "mean_z"
ykey = "cov_x__z"

I = It


def compare_drifting(I):
    fig, ax = plt.subplots()
    ax.scatter(pzstat(xkey), ptstat(ykey), label="particles at the same t")
    # ax.scatter(pzstat(xkey), pzstat(ykey), marker='x', label='particles at the same z')# (cov_x__t)*(-c)')
    ax.scatter(
        pzstat(xkey),
        pzstat("cov_x__t") * (-c_light),
        marker="x",
        label=r"particles at the same z: $\left<x, -ct\right>$",
    )
    ax.plot(
        I.stat(xkey),
        I.stat(ykey),
        label=f"fort.32, Flagdiag = {I.header['Flagdiag']} ",
        color="red",
    )
    plt.legend()
    ax.set_title(f"Flagdiag: {I.header['Flagdiag']}")
    ax.set_xlabel(xkey)
    ax.set_ylabel(ykey)

In [ ]:
compare_drifting(It)

In [ ]:
compare_drifting(Iz)

# Plot all 

In [ ]:
plt.rcParams["figure.max_open_warning"] = 50


def plot_all_cov(I):
    stats = I.output["stats"]
    for key in stats:
        if key.startswith("cov_"):
            I.plot(key)

## Flagdiag = 1 (same time)

In [ ]:
plot_all_cov(It)

## Flagdiag = 2 (drift to same z)

In [ ]:
plot_all_cov(Iz)